In [1]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import numpy as np
from datetime import date, datetime
import matplotlib.pyplot as plt
from string import Template
import os
import re
from collections import defaultdict
import random
import json
import asyncio
import nest_asyncio
nest_asyncio.apply()
%matplotlib inline


In [2]:
from datetime import datetime, timedelta

dateString = "2013/12/21"
dateFormatter = "%Y/%m/%d"
datetime.strptime(dateString, dateFormatter)
test_symbols = ['2330', '3008', '2317', '2412']
DEBUG = False

loop = asyncio.get_event_loop()

In [3]:
# async pipline Test brokerId fetch  
async def fetch_brokerId():
    URL = 'https://fubon-ebrokerdj.fbs.com.tw/z/js/zbrokerjs.djjs'
    
    resp = requests.get(URL)
    resp.encoding = 'cp950' # 中文
    return resp.text
  
async def post_brokerId():
    global BkrMap
    text =  await fetch_brokerId()
    reBkr = re.compile(r".*g_BrokerList = \'(.*)\'", re.MULTILINE|re.DOTALL)
    reBkr2Id = re.compile(r"(.*),(.*)", re.DOTALL)
    BkrGrps = reBkr.match(text).group(1).split(';') # slice to broker groups
    BkrMap = defaultdict(lambda: defaultdict(str))

    for bkrgrp in BkrGrps:
        bkrlist = bkrgrp.split('!') # slice to sub broker
        for i, bkr in enumerate(bkrlist):
            bmt = reBkr2Id.match(bkr)
            if bmt:
                top_bkr = bmt.group(2) if i == 0 else top_bkr
                BkrMap[top_bkr].update({bmt.group(2): bmt.group(1)}) #group(1) = b

    # debug            
    #for k in sorted(BkrMap.keys())[:1]:
    #    print(BkrMap[k])
loop = asyncio.get_event_loop()
loop.run_until_complete(post_brokerId())
#post_brokerId()

In [4]:
# random pick BHID, SBHIDS
def random_pick_BHIDs():
    global BkrMap
    test_BHIDS = list(BkrMap[random.choice(list(BkrMap.keys()))].items())
    print (test_BHIDS)
    BHID = test_BHIDS[0][1]
    SBHID = random.choice(test_BHIDS[1:])[1] if len(test_BHIDS) > 2 else BHID # if have sub SBHIDS
    #print (BHID, SBHID)
    return (BHID, SBHID)
    

In [5]:
BHID = '5380' # 台新
SBHID = '0035003300380041' # 台新建北

if not DEBUG:
    BHID, SBHID = random_pick_BHIDs()
    
print (BHID, SBHID)

[('大鼎(停)', '8770'), ('大鼎', '8770')]
8770 8770


In [6]:
#http://fubon-ebrokerdj.fbs.com.tw/z/zc/zco/zco0/zco0.djhtm?a=2330&BHID=9800&b=9800
# get broker stock position detail put/call volume
def get_broker_sub_symbol_position(BHID, SBHID, symbol):
    URLStr = Template('https://fubon-ebrokerdj.fbs.com.tw/z/zc/zco/zco0/zco0.djhtm?a=$SYMBOL&BHID=$BHID&b=$SBHID')
    URLPar = {
        'SYMBOL': symbol,
        'BHID' : BHID,
        'SBHID': SBHID
    }
    urlStr = URLStr.substitute(URLPar)
    resp = requests.get(urlStr)
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find_all('table', {'class': 't01'})[0]
    values = list(map(lambda x: str(x.text).replace(',',''), table.find_all('td', {'class': ['t3n1', 't3r1']})))
    ddates = list(map(lambda x: datetime.strptime(str(x.text), dateFormatter), table.find_all('td', {'class': 't4n0'})))
    #print (values)
    #print (len(values))
    calls = [values[i] for i in range(0, len(values), 4)]  #買
    puts = [values[i+1] for i in range(0, len(values), 4)] #賣
    BkrPutCall = {
        'date': ddates,
        'symbol': symbol,
        'puts': puts,
        'calls': calls
    }
    return  pd.DataFrame(BkrPutCall)

In [7]:
symbol = '2330'
if not DEBUG:
    symbol = random.choice(test_symbols)

BkrPutCallpf = get_broker_sub_symbol_position(BHID, SBHID, symbol)
BkrPutCallpf[:4]

,date,symbol,puts,calls


In [8]:
#https://fubon-ebrokerdj.fbs.com.tw/z/zg/zgb/zgb0.djhtm?a=1440&b=1440&c=E&d=1
def get_broker_all_symbol_position(BHID, SBHID):
    
    # parser GenLink2stk('AS3481','群創');
    reSymbol2Id = re.compile(r".*GenLink2stk\(\'AS(.*)\',\'(.*)\'\);", re.M|re.DOTALL)  
    
    #parser <a href="javascript:Link2Stk('00677U');">00677U富邦VIX</a>
    # no support yet
    #reSymbol2Id2 = re.compile(r".*Link2Stk\(\'(.*)\'\);\"\>(.*)\<\/a\>", re.M|re.DOTALL)
    
    def get_symbol(text): 
        smt = reSymbol2Id.match(text)
        if smt:
            return smt.group(1)
            
    def get_puts_calls(table, Bkrlist):
        values = list(map(lambda x: str(x.text).replace(',',''), table.find_all('td', {'class': ['t3n1', 't3r1']})))
        symbols = list(map(lambda x:  get_symbol(str(x.text)), table.find_all('td', {'class': 't4t1'})))
        
        for i, symbol in enumerate(symbols):
            if symbol == None:
                continue
            Bkrlist.append({
            'symbol': symbol,
            'calls': values[i*3+0], #買
            'puts': values[i*3+1]   #賣
            })
        
    URLStr = Template('https://fubon-ebrokerdj.fbs.com.tw/z/zg/zgb/zgb0.djhtm?a=$BHID&b=$SBHID&c=E&d=1')
    URLPar = {
        'BHID' : BHID,
        'SBHID': SBHID
    }
    urlStr = URLStr.substitute(URLPar)
    resp = requests.get(urlStr)
    soup = BeautifulSoup(resp.text, 'html.parser')
    
    tables = soup.find_all('table', {'class': 't0'})
    Bkrlist = list()
    get_puts_calls(tables[0], Bkrlist) # 買超
    get_puts_calls(tables[1], Bkrlist) # 賣超
    
    return pd.DataFrame(Bkrlist)

In [9]:
BHID2Symbolpf = get_broker_all_symbol_position(BHID, SBHID)
BHID2Symbolpf.head()

""


In [10]:
import asyncio
import motor.motor_asyncio
from datetime import date, datetime
import pprint
import sys

sys.path.insert(0, os.path.abspath('../'))
import abupy
from abupy.UtilBu.ABuDateUtil import str_to_datetime

NumExpr defaulting to 8 threads.


In [11]:
client = motor.motor_asyncio.AsyncIOMotorClient('mongodb://localhost:27017')
loop = asyncio.get_event_loop()

In [12]:
# spider cmd
#%scrapy crawl StockHisTraderSpider --logfile test.log
yy, mm, dd = date.today().isoformat().split("-")[0:3]
cdate = "{0}-{1}-{2}".format(yy, mm, dd)
#cdate = '2020-12-04'
dd = str_to_datetime(cdate)
symbol = '3008'

In [13]:
db = client['symbol-db']
collection = db['StockHisTrader']

async def do_find_StockHisTrader_one():
    document = await db['StockHisTrader'].find_one({'symbol': symbol, 'date': dd})
    pprint.pprint(document)
    

loop.run_until_complete(do_find_StockHisTrader_one())

{'_id': ObjectId('603f94367d4ed9ee08324b56'),
 'date': datetime.datetime(2021, 3, 3, 0, 0),
 'symbol': '3008',
 'traders': [{'call': 77.0, 'put': 2.0, 'tdname': '花旗環球', 'weight': 9.66},
             {'call': 2.0, 'put': 45.0, 'tdname': '台灣匯立證券', 'weight': 5.54},
             {'call': 73.0, 'put': 2.0, 'tdname': '凱基-三多', 'weight': 9.15},
             {'call': 46.0, 'put': 66.0, 'tdname': '摩根大通', 'weight': 2.58},
             {'call': 68.0, 'put': 26.0, 'tdname': '美林', 'weight': 5.41},
             {'call': 5.0, 'put': 21.0, 'tdname': '統一-新竹', 'weight': 2.06},
             {'call': 102.0, 'put': 80.0, 'tdname': '台灣摩根士丹利', 'weight': 2.84},
             {'call': 12.0, 'put': 27.0, 'tdname': '瑞士信貸', 'weight': 1.93},
             {'call': 22.0, 'put': 0.0, 'tdname': '第一金-安和', 'weight': 2.84},
             {'call': 1.0, 'put': 13.0, 'tdname': '日盛-嘉義', 'weight': 1.55},
             {'call': 28.0, 'put': 8.0, 'tdname': '凱基-台北', 'weight': 2.58},
             {'call': 0.0, 'put': 11.0, 'tdname': 

In [14]:
async def do_find_StockHisData_one():
    document = await db['StockHisData'].find_one({'symbol': symbol, 'date': dd})
    pprint.pprint(document)
    
loop.run_until_complete(do_find_StockHisData_one())

{'C': 3440.0,
 'H': 3440.0,
 'L': 3315.0,
 'O': 3345.0,
 'V': 878.0,
 '_id': ObjectId('603f9635ff42d22eb5bec54c'),
 'date': datetime.datetime(2021, 3, 3, 0, 0),
 'symbol': '3008'}


In [15]:
async def do_find_StockMargion_one():
    document = await db['StockHisMargin'].find_one({'symbol': symbol, 'date': dd})
    pprint.pprint(document)
    
loop.run_until_complete(do_find_StockMargion_one())

{'MagPurchBalance': 1594.0,
 'MagPurchBuy': 14.0,
 'MagPurchCashRepay': 0.0,
 'MagPurchLimit': 33535.0,
 'MagPurchRatio': 4.75,
 'MagPurchSell': 52.0,
 'ShortDivMagRatio': 4.2,
 'ShortSaleBalance': 67.0,
 'ShortSaleBuy': 1.0,
 'ShortSaleCashRepay': 0.0,
 'ShortSaleSell': 2.0,
 '_id': ObjectId('603fa3a8edf89f818da527e4'),
 'date': datetime.datetime(2021, 3, 3, 0, 0),
 'symbol': '3008'}


In [16]:

async def do_find_max_calls_symbol():
    # test find max count of call symbols at sample time window
    pipeline = [
        # add lookup hisstock filter min/max close and margin rate, total_calls/total_volume = total_weight
        {"$match": { "date": { "$lte": datetime.now()} } },
        {"$match": { "date": { "$gte": datetime.now() - timedelta(days=2) } } },
        {"$unwind": "$traders" },
        {"$group": { "_id": { "tdname": "$traders.tdname", "symbol": "$symbol"}, 
                    "sum_puts": { "$sum": "$traders.put" }, 
                    "sum_calls": { "$sum": "$traders.call" } }  },
        {"$sort": { "sum_calls": -1, "sum_puts": 1 } },
        {"$limit": 3 }
        ]
    collect = db['StockHisTrader'].aggregate(pipeline)
    async for cursor in collect: # clean cursor ptr and store to list
        pprint.pprint(cursor)
        

loop.run_until_complete(do_find_max_calls_symbol())        

{'_id': {'symbol': '2409', 'tdname': '瑞士信貸'},
 'sum_calls': 49097.0,
 'sum_puts': 20712.0}
{'_id': {'symbol': '2409', 'tdname': '美商高盛'},
 'sum_calls': 44620.0,
 'sum_puts': 8769.0}
{'_id': {'symbol': '2610', 'tdname': '美林'},
 'sum_calls': 33716.0,
 'sum_puts': 25231.0}


In [17]:
yy, mm, dd = date.today().isoformat().split("-")[0:3]
dd = 19
cdate = "{0}-{1}-{2}".format(yy, mm, dd)
today = str_to_datetime(cdate)

C_MIN, C_MAX = 50, 200 #close Min:Max
V_MAX = 200 # volume >= V_max
broker_sampleD = 30 # broker sample [n]Days
candidates = []

In [18]:

async def do_find_jump_spot_symbols():
    # test find max count of call symbols at sample time window
    pipeline = [
        # add lookup hisstock filter min/max close and margin rate, total_calls/total_volume = total_weight

      # find jump spot 
      {"$lookup": {
           "from": "StockHisData",
           "pipeline": [
              {"$match":{ "$expr": {"$and": [
                        { "$lte": ["$C", C_MAX ] },
                        { "$gte": ["$C", C_MIN ] },
                        { "$eq": ["$C", "$O"]},
                        { "$ne": ["$O", "$H"]},
                        { "$gte": ["$V", V_MAX]},
                        { "$eq" : ["$date", today]},
         #         {"$eq": ["$symbol", "8044"]}
                       ]}}},
              {"$project": {"_id": 0 } } # no _id to next pipeline
           ],
           "as": "p0_item"
      }},
      {"$unwind":"$p0_item"}, 
      {"$match":{"$expr":{"$eq":["$p0_item.symbol", "$symbol"]}}},
      {"$match":{"$expr":{"$and":[
          { "$lte": ["$date", datetime.now() ] },
          { "$gte": ["$date", datetime.now() - timedelta(days=broker_sampleD) ] }
      ]}}},
      {"$unwind": "$traders" },
      {"$group": { "_id": { "tdname": "$traders.tdname", "symbol": "$symbol"}, 
                    "sum_puts": { "$sum": "$traders.put" }, 
                    "sum_calls": { "$sum": "$traders.call" },
                    "tD_C": { "$first": "$p0_item.C"},
                    "tD_O": { "$first": "$p0_item.O"},
                    "tD_H": { "$first": "$p0_item.H"},
                    "tD_L": { "$first": "$p0_item.L"},
                    "tD_V": { "$first": "$p0_item.V"},
                    "tD":   { "$first": "$p0_item.date"},
                    "broker_sampleD": { "$first": broker_sampleD }
                 }},
      {"$sort": { "sum_calls": -1, "sum_puts": 1, "tD_C": -1, "tD_V": -1 } },  
      #{"$limit": 1 }     
      ]
    collect = db['StockHisTrader'].aggregate(pipeline)
    async for cursor in collect: # clean cursor ptr and store to list
        pprint.pprint(cursor)
        candidates.append(cursor)
        
loop.run_until_complete(do_find_jump_spot_symbols())

In [19]:
test_symbols = []
for it in candidates:
    if it['_id']['symbol'] not in test_symbols:
        test_symbols += [it['_id']['symbol']] 

test_symbols[:5]

[]

In [20]:
short_sampleD = 1000
long_sampleD = 30

yy, mm, dd = date.today().isoformat().split("-")[0:3]
dd = 19
cdate = "{0}-{1}-{2}".format(yy, mm, dd)
today = str_to_datetime(cdate)

short_endD = today #datetime.now()
short_startD = datetime.now() - timedelta(days=short_sampleD)    
long_endD = datetime.now()
long_startD = datetime.now() - timedelta(days=long_sampleD)  

In [21]:
async def get_symbol_df(symbol, startD, endD):
    
    pipeline = [
        {"$match":{ "$expr": {"$and": [
                { "$lte": ["$date", endD ] },
                { "$gte": ["$date", startD ] },
                { "$eq": ["$symbol", symbol ] }
            ]}}},
         {"$project" : { "_id": 0 } }
        ]
    collect = db['StockHisData'].aggregate(pipeline)
    items = await collect.to_list(100)
    df = pd.DataFrame(items)
    df = df.set_index('date')
    return df
 
df = loop.run_until_complete(get_symbol_df('2330', short_startD, short_endD))
df

,O,H,L,C,V,symbol
date,,,,,,
2021-02-05,638.0,641.0,631.0,632.0,5.4091e+07,2330
2021-02-18,664.0,665.0,656.0,660.0,5.4520e+07,2330
2021-02-19,656.0,657.0,647.0,652.0,5.1365e+07,2330
2021-02-22,660.0,662.0,650.0,650.0,3.7296e+07,2330
2020-08-03,420.5,424.0,415.0,416.0,7.7289e+07,2330
...,...,...,...,...,...,...
2020-12-11,517.0,517.0,505.0,516.0,5.1297e+07,2330
2020-12-14,512.0,513.0,508.0,508.0,3.0810e+07,2330
2020-12-15,507.0,510.0,504.0,504.0,4.3060e+07,2330


In [22]:
test_symbols = []
for candidate in candidates:
    symbol = candidate['_id']['symbol'] 
    tD = candidate['tD']
    if symbol in test_symbols:
        continue
    test_symbols += [symbol]
    short_df = loop.run_until_complete(get_symbol_df(symbol, short_startD, short_endD))
    #long_df = loop.run_until_complete(get_symbol_df(symbol, long_startD, long_endD))
    short_df['C'] = short_df['C'].astype(float)
    #print (short_df.index[-1])
    n_short_df = short_df.sort_values(by='C', ascending=False)
    if short_df.index[-1] in n_short_df.index[-3:-1]:
        print (symbol, short_df.index[-3:-1])
